In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load fine-tuned BERT model
model_path = "/kaggle/input/bert-toxic-epoch1/other/default/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()


Using device: cuda


2025-04-20 20:42:54.868485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745181775.056274      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745181775.111010      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [2]:
# Load comments to score
test_df = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv")
print("Sample:", test_df.head())

Sample:    comment_id                                               text
0      114890  "\n \n\nGjalexei, you asked about whether ther...
1      732895  Looks like be have an abuser , can you please ...
2     1139051  I confess to having complete (and apparently b...
3     1434512  "\n\nFreud's ideas are certainly much discusse...
4     2084821  It is not just you. This is a laundry list of ...


In [3]:
# Weight each output label according to severity - most notebooks used this
weights = weights = [.32, 1.82, .16, 1.5, .64, 1.5]  

def predict_score(text):
    inputs = tokenizer(text, truncation=True, padding="max_length", max_length=256, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.sigmoid(logits).squeeze().cpu().tolist()
    
    return sum(p * w for p, w in zip(probs, weights))


In [4]:
print("Scoring all test comments...")
test_df["score"] = test_df["text"].apply(predict_score)
print("Finished scoring.")

Scoring all test comments...
Finished scoring.


In [5]:
submission = test_df[["comment_id", "score"]]
submission.to_csv("submission.csv", index=False)
print("Saved submission.csv for Kaggle submission.")

Saved submission.csv for Kaggle submission.


In [6]:
import re

# Load validation pairs
val_df = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")

def clean_text(text):
    text = text.strip().lower()
    text = re.sub(r"http\\S+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\\s]", "", text)
    return text

test_df["clean_text"] = test_df["text"].apply(clean_text)
test_map = dict(zip(test_df["clean_text"], test_df["score"]))

val_df["less_toxic_clean"] = val_df["less_toxic"].apply(clean_text)
val_df["more_toxic_clean"] = val_df["more_toxic"].apply(clean_text)

def agree(row):
    a = test_map.get(row["less_toxic_clean"], None)
    b = test_map.get(row["more_toxic_clean"], None)
    if a is None or b is None:
        return None
    return int(a < b)

val_df["agreement"] = val_df.apply(agree, axis=1)
valid_agreement = val_df["agreement"].dropna().mean()

print("Validation agreement accuracy:", valid_agreement)


Validation agreement accuracy: 0.6933428605454329
